# <center>Deep RL - Target Network </center>

# 1. Setup

#### Make sure that all the required dependencies are installed

In [ ]:
import sys
!{sys.executable} -m pip install gym

In [ ]:
!git clone https://github.com/Tobias-Fischer/RVSS2022-RL.git

#### Import all dependencies

In [ ]:
# Import all required dependencies

import gym
import math
import random
import numpy as np
from collections import namedtuple
import copy
from itertools import count

import torch
import torch.nn as nn
import torch.optim as optim
import torch.autograd as autograd 


import matplotlib.pyplot as plt
# %matplotlib inline

import io
import base64
from IPython.display import HTML
from IPython.display import clear_output

import sys
import os
sys.path.insert(0, os.path.abspath('RVSS2022-RL/Support'))

# if gpu is to be used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

RUN_TRAINING = True

# 2. CartPole Environment

In this notebook, we will DQN to solve the Cart-Pole environment.

![](https://raw.githubusercontent.com/Tobias-Fischer/RVSS2022-RL/main/Support/images/cartPole.gif)

## State Space
The state space of this environment is defined by a 4-tuple where each value represents:

![](https://raw.githubusercontent.com/Tobias-Fischer/RVSS2022-RL/main/Support/images/cartPole_actionSpace.png)

## Action Space
There are just two possible discrete actions

<table>
<thead>
<tr>
<th>Num</th>
<th>Action</th>
</tr>
</thead>
<tbody>
<tr>
<td>0</td>
<td>Push cart to the left</td>
</tr>
<tr>
<td>1</td>
<td>Push cart to the right</td>
    </tr>
</table>

## Rewards

Reward is 1 for every step taken, including the termination step

## Other Considerations
- When choosing the starting state, all observations are assigned a uniform random value in [-0.05..0.05]
- An episode is considered as terminated if:
    - Pole Angle is more than 12 degrees.
    - Cart Position is more than 2.4 (center of the cart reaches the edge of the display).
    - Episode length is greater than 200.
    
    
**The environment is considered to be solved when the average return is greater than or equal to 195.0 over 100 consecutive trials.**

# 3. DQN Algorithm with Target Network

In this notebook we will extend the DQN with Replay Memory implementation we just saw by adding a target network. This revised version of the DQN algorithm is shown below

![](https://raw.githubusercontent.com/Tobias-Fischer/RVSS2022-RL/main/Support/images/DQN_algorithm.png)


### The main changes are:

- We have extended the Agent class' attributes to include 2 DQN networks instead of one (one target and one policy network)

- We have change the method ``get_next_q(.)`` so the q-values are computed using the target network instead of the policy network

- We have added a new method called ``transfer_parameters``(.). This method replaces the parameters of the target network with those of the policy network 

- We have modified the main loop to include instructions that call ``transfer_parameters``(.) after a predefined number of episodes

## Replay Buffer

We use the same Replay Buffer implementation

**Note**: This implementation of the ReplayMemory class was taken from [***Pytorch DQN tutorial***](https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html)

In [ ]:
# This tuple represents one observation in our environment
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward', 'done'))

class ReplayMemory:
    """
    A cyclic buffer of bounded size (capacity) that holds the transitions 
    observed recently. 
    
    It also implements a sample() method for selecting a random 
    batch of transitions for training.
    """
    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.position = 0

    def push(self, *args):
        """Saves a transition."""
        if len(self.memory) < self.capacity:
            self.memory.append(None)
        self.memory[self.position] = Transition(*args)
        self.position = (self.position + 1) % self.capacity

    def sample(self, batch_size):
        """Returns a minibatch of `Transition` randomly
        Args:
            batch_size (int): Size of mini-bach
        Returns:
            List[Transition]: Minibatch of `Transition`
        """
        return random.sample(self.memory, batch_size)

    def __len__(self):
        """Returns the length """
        return len(self.memory)

## DQN Network

Let us now define the Multi Layer Perceptron network that will be used as the function approximator for the action-value function (q-function)

In [ ]:
class DQN(nn.Module):
    """DQN Network
        Args:
        input_dim (int): `state` dimension.
        output_dim (int): Number of actions.
        hidden_dim (int): Hidden dimension in fully connected layer
    """
    def __init__(self, num_inputs=4, num_actions=2, hidden_dim=128):
        super(DQN, self).__init__()
        
        self.layers = nn.Sequential(
            nn.Linear(num_inputs, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, num_actions)
        )
        
    def forward(self, x):
        """Returns a Q_value
        Args:
            x (torch.Tensor): `State` 2-D tensor of shape (n, num_inputs)
        Returns:
            torch.Tensor: Q_value, 2-D tensor of shape (n, num_actions)
        """
        return self.layers(x)

## DQN Agent

This class contains the main steps of the Deep Q-learnig algorithm.


In [ ]:
class DQNAgent(object):
    """DQN Agent
    This class contains the main steps of the DQN algorithm
    
    Attributes:
    action_value_net (DQN): Function approximator for our action-value function (predictor)
    target_net (DQN): Function approximator for our target action-value function
    loss_fn (MSELoss): Criterion that measures the mean squared error (squared L2 norm) 
                       between each element of the predicted and target q-values.
    optimizer (Adam): Stochastic gradient optimize
    gamma (float): Discount factor
    """
    
    def __init__(self, seed=123, input_dim=4, output_dim=2, 
                 hidden_dim=128, gamma=0.99):
        """
        Define instance of DQNAgent
        Args:
        seed (int): Value used to initialize random number generator
        input_dim (int): `state` dimension.
        output_dim (int): Number of actions.
        hidden_dim (int): Hidden dimension in fully connected layer
        """
        torch.manual_seed(seed)
        self.action_value_net = DQN(input_dim, output_dim, hidden_dim).to(device)
                
        # We add a target network. Both the policy and target networks must start with same parameters
        self.target_net = DQN(input_dim, output_dim, hidden_dim).to(device)
        self.target_net.load_state_dict(self.action_value_net.state_dict())
        self.target_net.eval()
                        
        self.loss_fn = nn.MSELoss()
        self.optimizer = optim.Adam(self.action_value_net.parameters(), lr=0.001)
                
        self.gamma = torch.tensor(gamma).float().to(device)
        
    def get_action(self, state, action_space_dim, epsilon):
        """
        Select next action using epsilon-greedy policy
        Args:
        epsilon (float): Threshold used to decide whether a random or maximum-value action 
                         should be taken next
         Returns:
            int: action index
        """        
        with torch.no_grad():
            cur_q = self.action_value_net(torch.from_numpy(state).float().to(device))
        q_value, action = torch.max(cur_q, axis=0)
        action = action if torch.rand(1,).item() > epsilon else torch.randint(0, action_space_dim, (1,)).item()
        action = torch.tensor([action]).to(device)
        return action
    
    def get_next_q(self, state):
        """Returns Q_value for maximum valued action at each state s
        Args:
            x (torch.Tensor): `State` 2-D tensor of shape (n, num_inputs)
        Returns:
            torch.Tensor: Q_value, 1 tensor of shape (n)
        """
        with torch.no_grad():
            next_q = self.target_net(state)
        q, _ = torch.max(next_q, axis=1)
        return q
    
    def optimize(self, batch):
        """Computes `loss` and backpropagation
        Args:
            batch: List[Transition]: Minibatch of `Transition`
        Returns:
            float: loss value
        """
        
        state_batch = torch.stack(batch.state)
        action_batch = torch.stack(batch.action)
        reward_batch = torch.stack(batch.reward)
        next_state_batch = torch.stack(batch.next_state)
                
        # Compute a mask of non-final states and concatenate the batch elements
        # (a final state is the one after which the simulation ends)
        non_final_mask = torch.tensor(tuple(map(lambda s: s.item() is not True,
                                          batch.done)), device=device, dtype=torch.bool)
        non_final_next_states = torch.stack([s for i, s in enumerate(batch.next_state)
                                            if batch.done[i].item() is not True])

        # Compute predicted q-values
        predicted_q = self.action_value_net(state_batch).gather(1, action_batch).reshape(1,-1)
        
        # Compute expected values for non-terminal and terminal states (this is our TD target)
        target_q = torch.zeros(len(batch.state), device=device)
        target_q[non_final_mask] = self.get_next_q(non_final_next_states)
        expected_q = reward_batch.reshape(1,-1)+(self.gamma * target_q)
        
        # Compute loss 
        loss = self.loss_fn(expected_q, predicted_q)
        
        # Use loss to compute gradient and update policy parameters through backpropagation
        self.optimizer.zero_grad()
        loss.backward(retain_graph=True)
        self.optimizer.step()
                
        return loss.item()
    
    def transfer_parameters(self):
        """Transfer parameters from action-value to target network
        """
        # Parameter transfer from policy to target network
        self.target_net.load_state_dict(self.action_value_net.state_dict())
        self.target_net.eval()

In [ ]:
def plot(frame_idx, rewards, losses):
    """
    Plot evolution of rewards and losses during training
    Args:
         rewards (list): Cummulative rewards for episodes seen so far
         losses (list): Prediction error at each training step
    
    """
    clear_output(True)
    plt.figure(figsize=(20,5))
    plt.subplot(131)
    plt.title('Episodes %s.\nCummulative reward last 10 episodes: %s' % (frame_idx, np.mean(rewards[-10:])))
    plt.plot(rewards)
    plt.ylabel("Avg. cummulative reward")
    plt.xlabel("No. of episodes")
    plt.subplot(132)
    plt.title('MSE Loss')
    plt.ylabel("Avg. cummulative TD-loss")
    plt.xlabel("No. of episodes")
    plt.plot(losses)
    plt.show()

### Running Parameters

The parameter ``freq_sync`` defines how often parameters are transferred between networks. 

In [ ]:
# Define running hyper-parameters and epsilon training sequence

memory_capacity = 1000
batch_size = 64
env_name = "CartPole-v0"
num_episodes = 800
epsilon_start = 1.0
epsilon_end = 0.01
epsilon_decay = 200
gamma = 0.99
hidden_dim = 128
freq_sync = 10

epsilon_by_step = lambda frame_idx: epsilon_end + (epsilon_start - epsilon_end) * math.exp(-1. * frame_idx / epsilon_decay)


In [ ]:
fig, ax = plt.subplots(figsize=(4, 4))

ax.plot([epsilon_by_step(i) for i in range(num_episodes)])
ax.set_xlabel("Num. episodes")
ax.set_ylabel("Epsilon")

plt.show()

### Main Loop and Replay Buffer Control

This is the main loop of our DQN implementation. Here we generate the samples added to the replay memory and train the agent using a batch sampled for the replay memory

In [ ]:
losses_list, rewards_list, episode_len_list = [], [], []
list_epsilon = []
replay_buffer = ReplayMemory(memory_capacity)

env = gym.make(env_name)
n_actions = env.action_space.n
dim_state = env.observation_space.shape[0]

agent = DQNAgent(seed=1234, input_dim=dim_state, 
                 output_dim=n_actions, 
                 hidden_dim=hidden_dim)

cur_epsilon = epsilon_start

if RUN_TRAINING:
    for i_episode in range(num_episodes):

        state, done, ep_len, losses, rewards = env.reset(), False, 0, 0, 0

        cur_epsilon = epsilon_by_step(i_episode+1)
        list_epsilon += [cur_epsilon]
        while not done:
            ep_len += 1
            action = agent.get_action(state, n_actions, cur_epsilon)
            next_state, reward, done, _ = env.step(action.item())
            rewards += reward

            t_s = torch.tensor(state).float().to(device)
            t_r = torch.tensor([reward]).float().to(device)
            t_ns = torch.tensor(next_state).float().to(device)
            t_a = action.to(device)
            t_done = torch.tensor([done]).bool().to(device)

            replay_buffer.push(t_s, t_a, t_ns, t_r, t_done)
            state = next_state

            if len(replay_buffer) > batch_size:
                transitions = replay_buffer.sample(batch_size)
                batch = Transition(*zip(*transitions))
                loss = agent.optimize(batch)
                losses += loss

        losses_list.append(losses / ep_len)
        rewards_list.append(rewards)
        episode_len_list.append(ep_len)

        # Add rule that call transfer_parameters() every freq_sync episodes
        if i_episode % freq_sync == 0:
            agent.transfer_parameters()


        if i_episode % 20 == 0:
            plot(i_episode, rewards_list, losses_list)

### Let's now test our trained model

Can we do better than the random policy?

We can verify the performance of our agent by looking at the cummulative reward 

In [ ]:
from gym import wrappers

env = gym.make("CartPole-v0")
env = wrappers.Monitor(env, "./gym-results", force=True, video_callable=False)
n_trials = 100
list_rewards = []

for i in range(n_trials):
    observation = env.reset()
    ep_return = 0
    done = False
    while not done:
        action = agent.get_action(observation, n_actions, 0.01)
        observation, reward, done, info = env.step(action.item())
        ep_return += reward
    list_rewards.append(ep_return)
env.close()

fig, ax = plt.subplots(figsize=(4, 4))
ax.plot(list_rewards)
ax.set_xlabel("Num. Trial")
ax.set_ylabel("Return")
ax.set_title('Average return %s +- %s' % (np.round(np.mean(list_rewards),2), np.round(np.std(list_rewards), 2)))

plt.show()

In [ ]:
print(list_rewards)

### This is an example of the performance obtained after training the agent with the hyper-parameters listed above for a total of 500 episodes.

In [ ]:
import io
import base64
from IPython.display import HTML

video = io.open('./RVSS2022-RL/Support/images/exampleSolution.mp4', 'r+b').read()
encoded = base64.b64encode(video)
HTML(data='''
    <video width="360" height="auto" alt="test" controls><source src="data:video/mp4;base64,{0}" type="video/mp4" /></video>'''
.format(encoded.decode('ascii')))